# 데이콘 3d 숫자 이미지 분류하기

### https://dacon.io/competitions/official/235951/overview/description

## Private [81th]

In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 509 kB 7.6 MB/s 


In [ ]:
import h5py # .h5 파일을 읽기 위한 패키지
import random
import pandas as pd
import numpy as np
import os
import glob
from google.colab import drive

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

from sklearn.metrics import accuracy_score
import torchvision
import sys
import warnings
import torchvision.transforms as transforms
import random
warnings.filterwarnings(action='ignore') 

In [ ]:
import timm

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
CFG = {
    'EPOCHS':10,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE':16,
    'SEED':41
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sys.setrecursionlimit(10**7)

# 새 섹션

In [ ]:
all_df = pd.read_csv('/content/drive/MyDrive/open/train.csv')
all_points = h5py.File('/content/drive/MyDrive/open/train.h5','r')

train_df = all_df.iloc[:int(len(all_df)*0.8)]
val_df = all_df.iloc[int(len(all_df)*0.8):]

In [ ]:
import random as rand
import cv2 as cv

In [ ]:
idx = 44

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/open/train.csv')
train_all = h5py.File('/content/drive/MyDrive/open/train.h5','r')
train_all = [np.array(train_all[str(i)]) for i in tqdm(train_df["ID"])]

  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
train_all = pd.read_csv('/content/drive/MyDrive/open/train.csv')
train_df = [np.array(all_df[str(i)]) for i in tqdm(train_all["ID"])]

In [ ]:
def rotate(b, c, dots):
    my = np.array([[np.cos(b), 0, np.sin(b)], [0, 1, 0], [-np.sin(b), 0, np.cos(b)]])
    mz = np.array([[np.cos(c), -np.sin(c), 0], [np.sin(c), np.cos(c), 0], [0, 0, 1]])
    m = np.dot(my,mz)
    dots = np.dot(dots, m.T)
    return dots

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, id_list, label_list, point_list):

        self.id_list = id_list
        self.label_list = label_list
        self.point_list = point_list

        
    def __getitem__(self, index):
        image_id = self.id_list[index]
        
        # h5파일을 바로 접근하여 사용하면 학습 속도가 병목 현상으로 많이 느리다.
        points = self.point_list[str(image_id)][:]
        image = self.get_vector(points)
        
        if self.label_list is not None:
            label = self.label_list[index]
            return torch.Tensor(image).unsqueeze(0), label
        else:
            return torch.Tensor(image).unsqueeze(0)
    
    def get_vector(self, points, x_y_z=[16, 16, 16]):
        # 3D Points -> [16,16,16]
        xyzmin = np.min(points, axis=0) - 0.001
        xyzmax = np.max(points, axis=0) + 0.001

        diff = max(xyzmax-xyzmin) - (xyzmax-xyzmin)
        xyzmin = xyzmin - diff / 2
        xyzmax = xyzmax + diff / 2

        segments = []
        shape = []

        for i in range(3):
            # note the +1 in num 
            if type(x_y_z[i]) is not int:
                raise TypeError("x_y_z[{}] must be int".format(i))
            s, step = np.linspace(xyzmin[i], xyzmax[i], num=(x_y_z[i] + 1), retstep=True)
            segments.append(s)
            shape.append(step)

        n_voxels =  x_y_z[1] * x_y_z[2] * 3

        n_y = x_y_z[1]
        n_z = x_y_z[2]

        structure = np.zeros((len(points), 4), dtype=int)
        structure[:,0] = np.searchsorted(segments[0], points[:,0]) - 1
        structure[:,1] = np.searchsorted(segments[1], points[:,1]) - 1
        structure[:,2] = np.searchsorted(segments[2], points[:,2]) - 1

        structure[:,3] =  structure[:,0] + (structure[:,2] *  (n_y)) 

        vector = np.zeros(n_voxels)
        count = np.bincount(structure[:,3])
        vector[:len(count)] = count

        vector = vector.reshape(3, n_z, n_y)
        augmentation = random.randint(1,8)
        if augmentation == 2:
                vector = rotate(-np.pi/augmentation, 0, vector)
            
        elif augmentation == 3:
                vector = rotate(-np.pi/augmentation, 0, vector)
            
        elif augmentation == 4:
                vector = rotate(-np.pi/augmentation, 0, vector)
            
        elif augmentation == 5:
                vector = rotate(-np.pi/augmentation, 0, vector)
                
        elif augmentation == 6:
                vector = rotate(-np.pi/augmentation, 0, vector)
            
        elif augmentation == 7:
                vector = rotate(-np.pi/augmentation, 0, vector)

        return vector.squeeze()

    def __len__(self):
        return len(self.id_list)

In [ ]:
trans = transforms.Compose([transforms.ToTensor()
                        ,transforms.Resize((32, 32)),
])

In [ ]:
from torchsummary import summary

In [ ]:
summary(model, input_size = (3, 32, 32))

In [ ]:
train_dataset = CustomDataset(train_df['ID'].values, train_df['label'].values, all_points)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_df['ID'].values, val_df['label'].values, all_points)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
class Custom_model(nn.Module):
    def __init__(self):
        super(Custom_model,self).__init__()
        self.model =timm.create_model('vgg16',pretrained = True, num_classes = 10).to(device)
        self.transconv = nn.Sequential(
            nn.ConvTranspose2d(in_channels = 3, out_channels = 3, kernel_size = 3, stride = 1, bias = True),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels = 3, out_channels = 16, kernel_size = 3, stride = 2, padding = 2, bias = True),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels = 16, out_channels = 3, kernel_size = 3, stride = 2, padding = 2, bias = True),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels = 3, out_channels = 3, kernel_size = 2, stride = 2, padding = 2, bias = True),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels = 3, out_channels = 3, kernel_size = 2, stride = 2, padding = 2, bias = True),
            nn.ReLU()

        )
    def forward(self,x):
        x = x.squeeze()
        x = self.transconv(x)
        x = self.model(x)

        return x


In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    best_score = 0
    epochs = 4
    val_list = [0]
    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = []
        for data, label in tqdm(iter(train_loader)):
            data, label = data.float().to(device), label.long().to(device)
            optimizer.zero_grad()
            
            output = model(data)
            loss = criterion(output, label)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        if scheduler is not None:
            scheduler.step()
            
        val_loss, val_acc = validation(model, criterion, val_loader, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss)}] Val Loss : [{val_loss}] Val ACC : [{val_acc}]')
        val_list.append(val_acc)
        torch.save(model.state_dict(), '/content/drive/MyDrive/open/best_model.pth')

In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()
    true_labels = []
    model_preds = []
    val_loss = []
    with torch.no_grad():
        for data, label in tqdm(iter(val_loader)):
            data, label = data.float().to(device), label.long().to(device)
            
            model_pred = model(data)
            loss = criterion(model_pred, label)
            
            val_loss.append(loss.item())
            
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += label.detach().cpu().numpy().tolist()
    
    return np.mean(val_loss), accuracy_score(true_labels, model_preds)

In [ ]:
model = Custom_model()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = None

train(model, optimizer, train_loader, val_loader, scheduler, device)

In [ ]:
ffnnnmmm

In [ ]:
hhrkkkkkbbbjhjh

In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/open/best_model.pth')
model = Custom_model()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = None
model.load_state_dict(checkpoint)
train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [0.1672297607919434] Val Loss : [0.33300234980396926] Val ACC : [0.9011]


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

...

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : [3] Train Loss : [0.15498404669605662] Val Loss : [0.35264231291413306] Val ACC : [0.8947]


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : [4] Train Loss : [0.1459204523995053] Val Loss : [0.36267367355562746] Val ACC : [0.9026]


In [ ]:
fgㅇㄹㅇㄹ

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/open/sample_submission.csv')
test_points = h5py.File('/content/drive/MyDrive/open/test.h5', 'r')

In [ ]:
test_dataset = CustomDataset(test_df['ID'].values, None, test_points)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def predict(model, test_loader, device):
    model.to(device)
    model.eval()
    model_preds = []
    with torch.no_grad():
        for data in tqdm(iter(test_loader)):
            data = data.float().to(device)
            
            batch_pred = model(data)
            
            model_preds += batch_pred.argmax(1).detach().cpu().numpy().tolist()
    
    return model_preds

preds = predict(model, test_loader, device)

  0%|          | 0/2500 [00:00<?, ?it/s]

In [ ]:
test_df['label'] = preds
test_df.to_csv('/content/drive/MyDrive/open/submit2.csv', index=False)

## 3d 이미지를 만지는 방법을 몰라 가장 영향이 없을 거 같은 높이 차원을 없애 2d이미지로 변환해보았다. 최종 Accuracy : 0.53